# DATA301 Project

Research Question:  What is the worldwide sentiment of the upcoming 2022 FIFA world cup in comparison to both the host country Qatar’s sentiment of the tournament, and the worldwide sentiment of the 2018 competition? 

## setup libraries

In [ ]:
#library and code setup
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

!pip install gdelt

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.lau

In [ ]:
#start spark local server
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

## fetch gdelt summary data for each country

In [ ]:
# gdelt summary data for the 2022 Qatar tournament

host = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:QA%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
unitedstates = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:US%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
newzealand = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:NZ%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
germany = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:GM%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
brazil = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:BR%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
ghana = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:GH%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
iran = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:IR%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
southkorea = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:KS%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
senegal = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:SG%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
unitedkingdom = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:UK%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"
argentina = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:AR%20repeat2:%22qatar%22&startdatetime=20210521000000&enddatetime=20220521235959&mode=tonechart"


# gdelt summary data for the 2018 Russia tournament

rhost = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:RS%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
runitedstates = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:US%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rnewzealand = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:NZ%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rgermany = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:GM%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rbrazil = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:BR%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rghana = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:GH%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
riran = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:IR%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rsouthkorea = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:KS%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rsenegal = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:SG%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
runitedkingdom = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:UK%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"
rargentina = "https://api.gdeltproject.org/api/v2/doc/doc?format=csv&query=%22world%20cup%22%20sourcecountry:AR%20repeat2:%22russia%22&startdatetime=20170101000000&enddatetime=20180101235959&mode=tonechart"

In [ ]:
# these remove files from previous runs
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


In [ ]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os
import urllib.request

start_time = time.monotonic() 

# multiprocess the query
e = ProcessPoolExecutor(16)


def get_filename(x):
  # return the filename where x is the name of the source country
  return f"{x}_gdeltdata.csv"


def into_file(filename):
  # pull the data from the gdelt summary api and write it to the countries file
  try:

    if not os.path.exists(filename):
      country = filename.split("_")[0]
      query = csv_dict.get(country)

      with urllib.request.urlopen(query) as testfile, open(filename, 'w') as f:
        f.write(testfile.read().decode())

    return filename

  except Exception as inst:
    print(type(inst))    # the exception instance
    print(inst.args)     # arguments stored in .args
    print("Error occurred")


country_list = ['Host', 'unitedstates', 'newzealand', 'germany', 'brazil', 'ghana', 'iran', 'southkorea', 'senegal', 'unitedkingdom', 'argentina', 'rHost', 'runitedstates', 'rnewzealand', 'rgermany', 'rbrazil', 'rghana', 'riran', 'rsouthkorea', 'rsenegal', 'runitedkingdom', 'rargentina']
csv_dict = {'Host': host, 'unitedstates': unitedstates, 'newzealand': newzealand, 'germany':germany, 'brazil':brazil, 'ghana':ghana, 'iran':iran, 'southkorea':southkorea, 'senegal':senegal, 'unitedkingdom':unitedkingdom, 'argentina':argentina, 'rHost': rhost, 'runitedstates': runitedstates, 'rnewzealand': rnewzealand, 'rgermany':rgermany, 'rbrazil':rbrazil, 'rghana':rghana, 'riran':riran, 'rsouthkorea':rsouthkorea, 'rsenegal':rsenegal, 'runitedkingdom':runitedkingdom, 'rargentina':rargentina}

countries = [get_filename(x) for x in country_list]
results = list(e.map(into_file, countries))


print(results)

['Host_gdeltdata.csv', 'unitedstates_gdeltdata.csv', 'newzealand_gdeltdata.csv', 'germany_gdeltdata.csv', 'brazil_gdeltdata.csv', 'ghana_gdeltdata.csv', 'iran_gdeltdata.csv', 'southkorea_gdeltdata.csv', 'senegal_gdeltdata.csv', 'unitedkingdom_gdeltdata.csv', 'argentina_gdeltdata.csv', 'rHost_gdeltdata.csv', 'runitedstates_gdeltdata.csv', 'rnewzealand_gdeltdata.csv', 'rgermany_gdeltdata.csv', 'rbrazil_gdeltdata.csv', 'rghana_gdeltdata.csv', 'riran_gdeltdata.csv', 'rsouthkorea_gdeltdata.csv', 'rsenegal_gdeltdata.csv', 'runitedkingdom_gdeltdata.csv', 'rargentina_gdeltdata.csv']


## read downloaded files into RDDs

In [ ]:
from pyspark.sql import SQLContext, SparkSession

sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

frame_list = []

for country in results:
  data_url = sqlContext.read.option("header", "true").csv(country)
  country_name = country.split("_")[0]
  frame_list.append((data_url, country_name))

print(frame_list)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


[(DataFrame[Label: string, Count: string, TopArtURL1: string, TopArtTitle1: string, TopArtURL2: string, TopArtTitle2: string, TopArtURL3: string, TopArtTitle3: string, TopArtURL4: string, TopArtTitle4: string, TopArtURL5: string, TopArtTitle5: string, TopArtURL6: string, TopArtTitle6: string, TopArtURL7: string, TopArtTitle7: string, TopArtURL8: string, TopArtTitle8: string, TopArtURL9: string, TopArtTitle9: string, TopArtURL10: string, TopArtTitle10: string], 'Host'), (DataFrame[Label: string, Count: string, TopArtURL1: string, TopArtTitle1: string, TopArtURL2: string, TopArtTitle2: string, TopArtURL3: string, TopArtTitle3: string, TopArtURL4: string, TopArtTitle4: string, TopArtURL5: string, TopArtTitle5: string, TopArtURL6: string, TopArtTitle6: string, TopArtURL7: string, TopArtTitle7: string, TopArtURL8: string, TopArtTitle8: string, TopArtURL9: string, TopArtTitle9: string, TopArtURL10: string, TopArtTitle10: string], 'unitedstates'), (DataFrame[Label: string, Count: string, TopA

In [ ]:
def get_tones(row):
  return (row['Label'], row['Count'])

countries_tone = sc.emptyRDD()
russia_countries_tone = sc.emptyRDD()

for (frame, country) in frame_list:
  tones = frame.rdd.map(get_tones)
  if country[0] != 'r':    # the dataset for the qatar competition
    country_tones = tones.map(lambda x: (x, country))
    countries_tone = countries_tone.union(country_tones)
  else:
    country_name = country[1:]
    country_tones = tones.map(lambda y: (y, country_name))
    russia_countries_tone = russia_countries_tone.union(country_tones)

print(countries_tone.collect())
print(russia_countries_tone.collect())

[(('-8', '1'), 'Host'), (('-7', '1'), 'Host'), (('-6', '11'), 'Host'), (('-5', '3'), 'Host'), (('-4', '18'), 'Host'), (('-3', '29'), 'Host'), (('-2', '35'), 'Host'), (('-1', '52'), 'Host'), (('0', '194'), 'Host'), (('1', '284'), 'Host'), (('2', '392'), 'Host'), (('3', '395'), 'Host'), (('4', '258'), 'Host'), (('5', '239'), 'Host'), (('6', '149'), 'Host'), (('7', '96'), 'Host'), (('8', '57'), 'Host'), (('9', '14'), 'Host'), (('10', '14'), 'Host'), (('11', '6'), 'Host'), (('-12', '1'), 'unitedstates'), (('-11', '1'), 'unitedstates'), (('-10', '0'), 'unitedstates'), (('-9', '1'), 'unitedstates'), (('-8', '3'), 'unitedstates'), (('-7', '4'), 'unitedstates'), (('-6', '33'), 'unitedstates'), (('-5', '32'), 'unitedstates'), (('-4', '76'), 'unitedstates'), (('-3', '254'), 'unitedstates'), (('-2', '520'), 'unitedstates'), (('-1', '319'), 'unitedstates'), (('0', '382'), 'unitedstates'), (('1', '238'), 'unitedstates'), (('2', '131'), 'unitedstates'), (('3', '44'), 'unitedstates'), (('4', '33'), '

## Cosine Similarity Helper Functions


In [ ]:
import numpy

def add_country(tone_group):
  # add countries with a tone count of 0 to tone values which have no country
  count_country = tone_group[1]

  added_countries = []
  for i in count_country:
    added_countries.append(i[1])

  up_to = len(country_list) // 2
  for i in country_list[:up_to]:
    if i not in added_countries:
      tone_group[1].append((0, i))
  return tone_group

def country_as_key(tones):
  # rearrange so in (country (tone, count)) format
  reformat = []
  tone = tones[0]
  for i in tones[1]:
    count, country = i[0], i[1]
    reformat.append((country, (tone, count)))
  return reformat

def vector_country(country):
  count_array = numpy.array([0])
  for i in range(len(country[1])):
    count_array = numpy.append(count_array, country[1][i][1])
  return (country[0], count_array)

def multiple_vectors(country):
  q_array = numpy.array([0])
  r_array = numpy.array([0])
  for i in range(len(country[1])):
    q_array = numpy.append(q_array, country[1][i][1][0])
    r_array = numpy.append(r_array, country[1][i][1][1])
  return (country[0], (q_array, r_array))

def cos_similarity(country, host):
  v1 = country[1]
  v2 = host[0][1] 
  similarity = v1.dot(v2) / (numpy.linalg.norm(v1)*numpy.linalg.norm(v2))
  return country[0], similarity

def cos_similarity_2(country):
  v1 = country[1][0]
  v2 = country[1][1]
  similarity = v1.dot(v2) / (numpy.linalg.norm(v1)*numpy.linalg.norm(v2))
  return country[0], similarity

## Cosine Similarity for Qatar World Cup

In [ ]:
tonecount_ints = countries_tone.map(lambda a: (int(a[0][0]), (int(a[0][1]), a[1])))
tone_group = tonecount_ints.groupByKey().map(lambda x: (x[0], list(x[1])))

append_country = tone_group.map(add_country).sortByKey()
country_key = append_country.flatMap(country_as_key)
country_grouped = country_key.groupByKey().map(lambda y: (y[0], list(y[1])))
country_vectors = country_grouped.map(vector_country)
countries_sorted = country_vectors.sortByKey()
qatar = countries_sorted.take(1)

cosine_similarity = country_vectors.map(lambda z: cos_similarity(z, qatar))
ordered_similarity = cosine_similarity.sortBy(lambda s: s[1], False)

print(ordered_similarity.collect())


[('Host', 1.0), ('ghana', 0.8898294320269681), ('iran', 0.7900947401927789), ('brazil', 0.7457724210891178), ('argentina', 0.6711627993522642), ('southkorea', 0.6156322379182334), ('senegal', 0.538738991008716), ('newzealand', 0.501889892836966), ('unitedkingdom', 0.4804413109152161), ('unitedstates', 0.42952514643079415), ('germany', 0.3957754423612963)]


## Cosine Similarity for Russia World Cup

In [ ]:
r_tonecount_ints = russia_countries_tone.map(lambda a: (int(a[0][0]), (int(a[0][1]), a[1])))
r_tone_group = r_tonecount_ints.groupByKey().map(lambda x: (x[0], list(x[1])))

r_append_country = r_tone_group.map(add_country).sortByKey()
r_country_key = r_append_country.flatMap(country_as_key)
r_country_grouped = r_country_key.groupByKey().map(lambda y: (y[0], list(y[1])))
r_country_vectors = r_country_grouped.map(vector_country)
r_countries_sorted = r_country_vectors.sortByKey()
r_russia = r_countries_sorted.take(1)

r_cosine_similarity = r_country_vectors.map(lambda z: cos_similarity(z, r_russia))
r_ordered_similarity = r_cosine_similarity.sortBy(lambda s: s[1], False)

print(r_ordered_similarity.collect())

[('Host', 1.0000000000000002), ('unitedstates', 0.9647703769109914), ('southkorea', 0.9600129845598563), ('germany', 0.9578561804849589), ('brazil', 0.9477209615954832), ('ghana', 0.925659835092976), ('senegal', 0.9250386998736069), ('argentina', 0.887595086084144), ('newzealand', 0.8783321556692931), ('unitedkingdom', 0.8712505590122548), ('iran', 0.7599014465688041)]


## Cosine Similarity between the World Cups

In [ ]:
reformat = country_key.map(lambda x: ((x[0], x[1][0]), x[1][1]))
r_reformat = r_country_key.map(lambda y: ((y[0], y[1][0]), y[1][1]))
joined_countries = reformat.join(r_reformat)
joined_reformat = joined_countries.map(lambda z: (z[0][0], (z[0][1], z[1])))
join_grouped = joined_reformat.groupByKey().map(lambda t: (t[0], list(t[1])))
join_vectors = join_grouped.map(multiple_vectors)

join_cosine_similarity = join_vectors.map(cos_similarity_2)
ordered_similarity = join_cosine_similarity.sortBy(lambda s: s[1], False)

end_time = time.monotonic()

print(ordered_similarity.collect())
print("processing took ",end_time-start_time)

[('argentina', 0.9739663796651391), ('southkorea', 0.967170723135395), ('senegal', 0.9654351668761568), ('unitedstates', 0.9489108161819158), ('brazil', 0.9415620382798787), ('germany', 0.9299038175434822), ('unitedkingdom', 0.8694416735055177), ('iran', 0.7696097340598517), ('ghana', 0.760090964494402), ('newzealand', 0.7525238433640895), ('Host', 0.6240243284009356)]
processing took  41.46170193499984
